[ResNet of roboflow](https://colab.research.google.com/github/captcha-challenger/hcaptcha-model-factory/blob/main/automation/roboflow_resnet.ipynb)

In [ ]:
from __future__ import annotations

!nvidia-smi

In [ ]:
!pip install loguru onnx fire hcaptcha-challenger
!git clone https://github.com/captcha-challenger/hcaptcha-model-factory.git
!mv -f hcaptcha-model-factory/src .
!rm -rf hcaptcha-model-factory/

Set `task_name` and `onnx_archive_name`

In [ ]:
import os

import hcaptcha_challenger as solver

os.environ["GITHUB_TOKEN"] = ""
task_name = "robot"
onnx_archive_name = "robot2312"

solver.diagnose_task(task_name)

Upload zip type datasets to `[PROJECT_DIR]/`

In [ ]:
from pathlib import Path
from loguru import logger
import zipfile

this_dir = Path("/content")
project_dir = this_dir

model_dir = project_dir.joinpath("model")
factory_data_dir = project_dir.joinpath("data")
source_dir = project_dir.joinpath("src")
zip_path = project_dir.joinpath(f"{task_name}.zip")

if not zip_path.exists():
    raise FileNotFoundError
with zipfile.ZipFile(zip_path) as z:
    z.extractall(factory_data_dir.joinpath(task_name))

Training

In [ ]:
%cd {source_dir}

from factories.resnet import ResNet


# - INPUT: `[PROJECT]/data/<task_name>`
# - OUTPUT: `[PROJECT]/model/<task_name>/<task_name.onnx>`

model = ResNet(
    task_name=task_name,
    epochs=None,  # default to 200
    batch_size=None,  # default to 4
    dir_dataset=str(factory_data_dir),
    dir_model=str(model_dir),
)
model.train()
model.conv_pth2onnx(verbose=False)

Deployment model to GitHub

In [ ]:
import locale

locale.getpreferredencoding = lambda d=True: "UTF-8"

!pip install PyGithub

import shutil
import sys
from github import Auth, Github, GithubException


def quick_development():
    auth = Auth.Token(os.getenv("GITHUB_TOKEN", ""))
    repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")
    modelhub_title = "ONNX ModelHub"

    model_path = model_dir.joinpath(task_name, f"{task_name}.onnx")
    pending_onnx_path = model_dir.joinpath(task_name, f"{onnx_archive_name}.onnx")
    shutil.copy(model_path, pending_onnx_path)

    for release in repo.get_releases():
        if release.title != modelhub_title:
            continue
        try:
            asset = release.upload_asset(path=str(pending_onnx_path))
        except GithubException as err:
            if err.status == 422:
                logger.error(
                    f"The model file already exists, please manually replace the file with the same name - url={repo.releases_url}",
                    url=repo.releases_url,
                )
        except Exception as err:
            logger.error(err)
        else:
            logger.success(
                f"Model file uploaded successfully "
                f"- name={asset.name} url={asset.browser_download_url}"
            )
            return asset.id


if not os.getenv("GITHUB_TOKEN"):
    logger.warning("Skip model deployment, miss GITHUB TOKEN")
    sys.exit()

aid = quick_development()
aid

Rolling upgrade

In [ ]:
import inspect
import os
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, List

import yaml
from github.GitReleaseAsset import GitReleaseAsset
from github.Repository import Repository
from hcaptcha_challenger import ModelHub
import hcaptcha_challenger as solver


@dataclass
class Objects:
    branches: Dict[str, Any]
    circle_seg: str
    nested_categories: Dict[str, List[str]]
    ashes_of_war: Dict[str, Any]
    label_alias: Dict[str, Any]
    datalake: dict

    @classmethod
    def from_modelhub(cls, modelhub: ModelHub):
        data = yaml.safe_load(modelhub.objects_path.read_text(encoding="utf8"))
        return cls(
            **{
                key: (data[key] if val.default == val.empty else data.get(key, val.default))
                for key, val in inspect.signature(cls).parameters.items()
            }
        )

    def to_yaml(self, path: Path | None = None):
        path = path or Path("objects-tmp.yaml")
        with open(path, "w", encoding="utf8") as file:
            yaml.safe_dump(self.__dict__, file, sort_keys=False, allow_unicode=True)
        return path

    @staticmethod
    def to_asset(repo: Repository, data_tmp_path: Path, message: str = ""):
        content = data_tmp_path.read_bytes()
        message = message or f"Automated deployment @ utc {datetime.utcnow()}"
        remote_path = "src/objects.yaml"
        sha = repo.get_contents(path=remote_path).sha
        return repo.update_file(
            branch="main", path=remote_path, message=message, content=content, sha=sha
        )


class Annotator:
    auth = Auth.Token(os.getenv("GITHUB_TOKEN"))
    repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")

    def __init__(self, asset_id: int, matched_label: str = ""):
        self._asset_id = asset_id
        self._matched_label = matched_label

        self._asset: GitReleaseAsset | None = None

        solver.install(upgrade=True)

        self.modelhub = ModelHub.from_github_repo()
        self.modelhub.parse_objects()

        self.data: Objects = Objects.from_modelhub(modelhub=self.modelhub)

    @property
    def asset(self):
        if not self._asset:
            self._asset = self.repo.get_release_asset(self._asset_id)
        return self._asset

    @staticmethod
    def parse_resnet_label(asset_name: str) -> str | None:
        """
        asset_name: dog2312.onnx chess_piece2309.onnx
        """
        onnx_archive = asset_name.replace(".onnx", "")
        i_end = -1
        for i, s in enumerate(onnx_archive):
            if s.isdigit():
                i_end = i
                break
        label = onnx_archive[:i_end]
        label = label.replace("_", " ")
        return label

    def handle_resnet_objects(self):
        onnx_archive = self.asset.name.replace(".onnx", "")
        matched_label = self._matched_label or self.parse_resnet_label(self.asset.name)
        old_onnx_archive = self.modelhub.label_alias.get(matched_label)

        # Match: create new case
        if not old_onnx_archive:
            self.data.label_alias[onnx_archive] = {"en": [matched_label]}
        # Match: update old case
        else:
            i18n_mapping = self.data.label_alias[old_onnx_archive].copy()
            del self.data.label_alias[old_onnx_archive]
            self.data.label_alias[onnx_archive] = i18n_mapping

    def flush_remote_objects(self):
        data_tmp_path = self.data.to_yaml()

        res = self.data.to_asset(
            self.repo,
            message=f"ci(annotator): update model `{self.asset.name}`",
            data_tmp_path=data_tmp_path,
        )

        logger.success(f"upgrade objects", response=res)

        os.remove(data_tmp_path)

    def execute(self):
        logger.debug(f"capture asset", name=self.asset.name, url=self.asset.browser_download_url)

        # Match: ResNet MoE models
        if "yolov8" not in self.asset.name:
            self.handle_resnet_objects()
            self.flush_remote_objects()


def roll_upgrade(asset_id):
    if not asset_id:
        return

    try:
        annotator = Annotator(asset_id)
        annotator.execute()
    except Exception as err:
        logger.warning(err)


if not os.getenv("GITHUB_TOKEN"):
    logger.warning("Skip the rolling upgrade task, miss GITHUB TOKEN")
    sys.exit()

roll_upgrade(asset_id=aid)